In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from keras import losses


2024-10-02 21:44:28.422050: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 21:44:28.429742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 21:44:28.438752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 21:44:28.441391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 21:44:28.447838: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727894669.483403   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894669.499720   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894669.499763   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))
sym_to_idx = {}
idx_to_sym = {}

for idx, sym in enumerate(alphabet):
    sym_to_idx[sym] = idx
    idx_to_sym[idx] = sym
    
text_idx = np.array([sym_to_idx[char] for char in text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)

dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727894669.928157   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894669.928208   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894669.928220   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894670.004623   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727894670.004670   33026 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-02

In [16]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 128),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
    keras.layers.Dense(len(alphabet), activation='relu')
])
model.summary()

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=10, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.4804 - loss: 2.4427
Epoch 2/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9591 - loss: 0.2631
Epoch 3/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.9628 - loss: 0.1926
Epoch 4/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9631 - loss: 0.1785
Epoch 5/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9671 - loss: 0.1585
Epoch 6/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9672 - loss: 0.1529
Epoch 7/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9675 - loss: 0.1506
Epoch 8/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9675 - loss: 0.1492
Epoch 9/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9676 - loss: 0.1485
Epoch 10/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9696 - loss: 0.1404


In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ''.join(res)
    return generated

In [17]:
print(predict_next(
    sample='б',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=200,
    temperature=0.6,
    batch_size=BATCH_SIZE
))

бо во бов ви по во вовосто Ио мемухововопо оли во и пы е, су дитото сто новя оме ппо, сь во воствогоежетоде ногодеже, вомегоде о намоби потовежего веготв за ва ко воно на води и сенимужегоднаето вестог


In [9]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=100,
    temperature=0.2,
    batch_size=BATCH_SIZE
))

1щшёP2»Бь0т60*е?э;БшhёiЖ-л@ЧeбохХв_пiАУ?b)4hгоТЛ06руn—Т«М(к,:вР#лiЯоЖэЗРнЭиУ3*oРш0aябИ, uА:йЬ._гпт#р1
